In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import random
import os
import glob
import numpy as np
import pandas as pan
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<em>This is Image Classification Task that I do not fully understand but I'm going to do because I'm now putting all my self worth on kaggle <em/>

In [ ]:
DATA_DIR = "working/augmentedData"

# Walk all the files and add them to this
all_files = [i for i in glob.glob(DATA_DIR + "//*//*")]
# Shuffle
random.shuffle(all_files)
# Get class names
all_labels = [os.path.dirname(i).split("/")[-1] for i in all_files]

In [ ]:
#make data frame, then do some EDA? #someone can fork this an do some good stuff with it though
zipped_data = list(zip(all_files, all_labels))
data_frame = pan.DataFrame(zipped_data, columns = ["Links", "Label"])

In [ ]:
#severe class imbalance; undersample majority class (which is literally a white patch)
data_frame["Label"].value_counts()
#lets sample 400 images of everything and concat to a new df
balance_df = pan.DataFrame()
large_labels = ["good", "thread error", "objects", "oil spot"]
for i in large_labels:
    balance_df = pan.concat([data_frame[data_frame["Label"] == i], balance_df], axis = 0)
balance_df = pan.concat([data_frame[data_frame["Label"] == "hole"], balance_df])#concat seperately as it has only 300 elems
balance_df["Label"].value_counts()

In [ ]:
#first see one of the images
test_arr = plotter.imread(balance_df.iloc[675]["Links"])
print(balance_df.iloc[675]["Label"])
plotter.imshow(test_arr)

In [ ]:
sns.countplot(x = balance_df["Label"]) #MASSIVE CLASS IMBALANCE GONE LOL

In [ ]:
#perform train_test_split
#label encode
labeller = dict(zip([i for i in balance_df["Label"].unique()], [i for i in range(len(balance_df["Label"].unique()))]))
features = balance_df["Links"]
label = balance_df["Label"].map(labeller)

feature_train, feature_eval, label_train, label_eval = train_test_split(features, label, test_size = 0.2, random_state = 101)
labeller

In [ ]:
# vals = label.value_counts().apply(lambda x: (x/len(label)) ** -1)
# keys = label.value_counts().index

# weights = dict(zip(keys, vals))

In [ ]:
#define dataset objects and preprocessors
feature_train_dataset = tf.data.Dataset.from_tensor_slices(feature_train.values)
feature_test_dataset = tf.data.Dataset.from_tensor_slices(feature_eval.values)
label_train_dataset = tf.data.Dataset.from_tensor_slices(label_train)
label_test_dataset = tf.data.Dataset.from_tensor_slices(label_eval)


def preprocessor(image):
    image = tf.io.read_file(image)
    image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, (64, 64))
    image = image/255
    return image

feature_train_dataset = feature_train_dataset.map(preprocessor)
feature_test_dataset = feature_test_dataset.map(preprocessor)

train = tf.data.Dataset.zip((feature_train_dataset, label_train_dataset))
test = tf.data.Dataset.zip((feature_test_dataset, label_test_dataset))

train = train.batch(128).shuffle(128).prefetch(tf.data.AUTOTUNE)
test = test.batch(256).prefetch(tf.data.AUTOTUNE)

In [ ]:
feature_train_dataset

In [ ]:
def build_model(learning_rate):
    model = Sequential()
    
    model.add(Conv2D(filters=32, kernel_size=(2, 2), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(filters=64, kernel_size=(2, 2), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(filters=128, kernel_size=(2, 2), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(filters=256, kernel_size=(2, 2), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    # model.add(Conv2D(filters=512, kernel_size=(2, 2), activation="relu"))
    # model.add(BatchNormalization())
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.5))

    # model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    # model.add(Dense(units=1024, activation="relu"))
    # model.add(Dropout(0.5))
    model.add(Dense(units=512, activation="relu"))
    model.add(Dropout(0.5))
    # model.add(Dense(units=256, activation="relu"))
    # model.add(Dropout(0.5))
    model.add(Dense(units=5, activation="softmax"))

    model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer=Adam(learning_rate=learning_rate))
    return model

model = build_model(learning_rate=0.0001)
model.build((None, 64, 64, 3))
model.summary()


In [ ]:
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(train, validation_data=test, callbacks=[early_stopping], epochs=200)

plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy over Epochs')
plt.show()

In [ ]:
# Evaluate the model
test_predictions = model.predict(test)
test_predictions_labels = np.argmax(test_predictions, axis=1)

# Calculate F1 score, precision, recall, and confusion matrix
f1 = f1_score(test_labels, test_predictions_labels, average='macro')
precision = precision_score(test_labels, test_predictions_labels, average='macro')
recall = recall_score(test_labels, test_predictions_labels, average='macro')
conf_matrix = confusion_matrix(test_labels, test_predictions_labels)

print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Confusion Matrix:\n{conf_matrix}")

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()